## Text classification

In [1]:
import pandas as pd
import numpy as np

In [28]:
#pip install -U imbalanced-learn

In [80]:
# Oversampling and under sampling
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

In [28]:
combined_df = pd.read_csv("combined_profile.csv")

In [49]:
pd.set_option("max_colwidth", None)

In [50]:
combined_df.head(10)


,username,age,occupation,status,gender,filtered sentence,y
0,123canwe,66.0,Retired,single,male,I full fire friskier hell,0
1,123WILFREDO,28.0,ESTUDIANTE,single,male,ME GUSTA CONOCER CHICAS BONDADOSAS CARIÑOSAS Y MUY FUERTES EN LA MORAL,0
2,1907,48.0,Construction,single,male,laid back earth good sense humor,0
3,52Jim52,70.0,Retired,divorced,male,Retired owner aerospace consuloting firm,0
4,Aaron90,28.0,Social worker,single,male,Hello aaron Vermont I vermont 2 years Im really fun person I like camping bone fire etc,0
5,abdelghani,71.0,retired,divorced,male,problem free flexibleseek love happiness,0
6,Abdul99,54.0,Executive,separated,male,Easy going Highly educated Life beautiful Travels,0
7,abdulrahmanwaly,33.0,journalist,single,male,name Abdulrahman person ethics religion work field journalism live Cairo,0
8,Abou,33.0,Designer,single,male,I serious honest trustworthy like share love fan culture new different person,0
9,Accydave,73.0,Retired,divorced,male,My name David Im 71 years old divorced retired 3 grownup children son 2 daughters,0


In [66]:
# features
X = combined_df['filtered sentence']

# target
y = combined_df['y']

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 424)

# output for slides

In [10]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(X)
xtrain_tfidf =  tfidf_vect.transform(X_train)
xvalid_tfidf =  tfidf_vect.transform(X_test)

In [58]:
XX = combined_df['filtered sentence'].iloc[2:5]

In [60]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(XX)
xtrain_tfidf =  tfidf_vect.transform(XX)


In [63]:
print(tfidf_vect.get_feature_names())  
print('\n')
print(xtrain_tfidf.shape)
print('\n')
print(xtrain_tfidf)

['2', 'aaron', 'aerospace', 'back', 'bone', 'camping', 'consuloting', 'earth', 'etc', 'fire', 'firm', 'fun', 'good', 'hello', 'humor', 'i', 'im', 'laid', 'like', 'owner', 'person', 'really', 'retired', 'sense', 'vermont', 'years']


(3, 26)


  (0, 23)	0.4082482904638631
  (0, 17)	0.4082482904638631
  (0, 14)	0.4082482904638631
  (0, 12)	0.4082482904638631
  (0, 7)	0.4082482904638631
  (0, 3)	0.4082482904638631
  (1, 22)	0.4472135954999579
  (1, 19)	0.4472135954999579
  (1, 10)	0.4472135954999579
  (1, 6)	0.4472135954999579
  (1, 2)	0.4472135954999579
  (2, 25)	0.21821789023599233
  (2, 24)	0.43643578047198467
  (2, 21)	0.21821789023599233
  (2, 20)	0.21821789023599233
  (2, 18)	0.21821789023599233
  (2, 16)	0.21821789023599233
  (2, 15)	0.43643578047198467
  (2, 13)	0.21821789023599233
  (2, 11)	0.21821789023599233
  (2, 9)	0.21821789023599233
  (2, 8)	0.21821789023599233
  (2, 5)	0.21821789023599233
  (2, 4)	0.21821789023599233
  (2, 1)	0.21821789023599233
  (2, 0)	0.2182178902359923

In [65]:
print(xtrain_tfidf.toarray())


[[0.         0.         0.         0.40824829 0.         0.
  0.         0.40824829 0.         0.         0.         0.
  0.40824829 0.         0.40824829 0.         0.         0.40824829
  0.         0.         0.         0.         0.         0.40824829
  0.         0.        ]
 [0.         0.         0.4472136  0.         0.         0.
  0.4472136  0.         0.         0.         0.4472136  0.
  0.         0.         0.         0.         0.         0.
  0.         0.4472136  0.         0.         0.4472136  0.
  0.         0.        ]
 [0.21821789 0.21821789 0.         0.         0.21821789 0.21821789
  0.         0.         0.21821789 0.21821789 0.         0.21821789
  0.         0.21821789 0.         0.43643578 0.21821789 0.
  0.21821789 0.         0.21821789 0.21821789 0.         0.
  0.43643578 0.21821789]]


# Naive Bayes classifier
- Taken from: https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

In [79]:
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#WITHOUT OVERSAMPLING/UNDER SAMPLING
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])
text_clf = text_clf.fit(X_train, y_train)

In [12]:
predictions = text_clf.predict(X_test)

### Evaluating the NB classifier

In [13]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("original dataset")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

original dataset
Accuracy: 0.9262295081967213
Precision: 0.8294993234100135
recall: 1.0
f1_score: 0.9068047337278106


In [69]:
#random oversampling
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(X)
xtrain_tfidf =  tfidf_vect.transform(X_train)
xvalid_tfidf =  tfidf_vect.transform(X_test)

text_clf = MultinomialNB()
ros = RandomOverSampler(random_state=42)

X_train_ros, y_train_ros= ros.fit_resample(xtrain_tfidf, y_train)
text_clf = text_clf.fit(X_train_ros, y_train_ros)
predictions = text_clf.predict(xvalid_tfidf)

In [70]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("random oversampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

random oversampling
Accuracy: 0.9268149882903981
Precision: 0.8306233062330624
recall: 1.0
f1_score: 0.9074759437453738


In [71]:
#random undersampling
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus= rus.fit_resample(xtrain_tfidf, y_train)
text_clf = MultinomialNB()
text_clf = text_clf.fit(X_train_rus, y_train_rus)
predictions = text_clf.predict(xvalid_tfidf)

In [72]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("random undersampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

random undersampling
Accuracy: 0.9256440281030445
Precision: 0.8283783783783784
recall: 1.0
f1_score: 0.9061345158906134


# SVM classifier

In [73]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=42)),
])

text_clf_svm = text_clf_svm.fit(X_train, y_train)

In [74]:
predictions = text_clf_svm.predict(X_test)

### Evaluating the SVM classifier

In [75]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

Accuracy: 0.9227166276346604
Precision: 0.8228187919463087
recall: 1.0
f1_score: 0.9027982326951398


In [76]:
#random oversampling
text_clf_svm = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42)

ros = RandomOverSampler(random_state=42)

X_train_ros, y_train_ros= ros.fit_resample(xtrain_tfidf, y_train)
text_clf_svm = text_clf_svm.fit(X_train_ros, y_train_ros)
predictions = text_clf_svm.predict(xvalid_tfidf)

In [77]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("oversampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

oversampling
Accuracy: 0.9221311475409836
Precision: 0.82171581769437
recall: 1.0
f1_score: 0.9021339220014717


In [78]:
#random undersampling
text_clf_svm = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42)

rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus= rus.fit_resample(xtrain_tfidf, y_train)
text_clf_svm = text_clf_svm.fit(X_train_rus, y_train_rus)
predictions = text_clf_svm.predict(xvalid_tfidf)

In [79]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("undersampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

undersampling
Accuracy: 0.9221311475409836
Precision: 0.82171581769437
recall: 1.0
f1_score: 0.9021339220014717


# Logistic regression

In [86]:
from sklearn.linear_model import LogisticRegression
# Train model
clf_lg = LogisticRegression()  
clf_lg = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf',LogisticRegression()),
])
clf_lg = clf_lg.fit(X_train, y_train) 

In [81]:
lg_predictions = clf_lg.predict(X_test)

### Evaluating logistic regression classifier

In [71]:
conf_matrix = confusion_matrix(y_test, lg_predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

Accuracy: 0.9106641721234799
Precision: 0.7995283018867925
recall: 0.9699570815450643
f1_score: 0.8765352294764058


In [87]:
#random oversampling
clf_lg = LogisticRegression()  

ros = RandomOverSampler(random_state=42)

X_train_ros, y_train_ros= ros.fit_resample(xtrain_tfidf, y_train)
clf_lg = clf_lg.fit(X_train_ros, y_train_ros)
lg_predictions = clf_lg.predict(xvalid_tfidf)

In [88]:
conf_matrix = confusion_matrix(y_test, lg_predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("oversampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

oversampling
Accuracy: 0.9320843091334895
Precision: 0.8446601941747572
recall: 0.9934747145187602
f1_score: 0.9130434782608695


In [89]:
#random undersampling
clf_lg = LogisticRegression()  

ros = RandomUnderSampler(random_state=42)

X_train_rus, y_train_rus= rus.fit_resample(xtrain_tfidf, y_train)
clf_lg = clf_lg.fit(X_train_rus, y_train_rus)
lg_predictions = clf_lg.predict(xvalid_tfidf)

In [90]:
conf_matrix = confusion_matrix(y_test, lg_predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("undersampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

undersampling
Accuracy: 0.9250585480093677
Precision: 0.8272604588394062
recall: 1.0
f1_score: 0.9054652880354505


# Reading real profile

In [3]:
import json
import os
import glob
import pandas as pd

In [53]:
import json
import os
import glob
import pandas as pd

json_dir = 'data/realprofile'
json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
realdf = pd.concat(dfs)

#adding a new column based on username column - to match with xxx.jpg when combining image captions
realdf["images"] = realdf["username"] + ".jpg"

#label
realdf["y"] = 1

In [71]:
scamdf[scamdf["images"] == "martinlawrence2.jpg"]

,year_reported,month_reported,username,name,age,occupation,status,inet,description,email,tags,gender,images,site,phone,y


# Reading scam profile

In [54]:
json_dir = 'data/scamprofile'
json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
scamdf = pd.concat(dfs)

#modifying the column "images" to join on filename - images = [xxx.jpg], change to xxx.jpg
scamdf["images"] = scamdf["images"].apply(lambda x: ''.join(x))

#label
scamdf["y"] = 0

# Reading image caption json (real)

In [55]:
json_dir = 'final caption model/xinyi_real'
json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

fn_list = []
caption_list = []
for file in file_list:
    with open(file) as f:
        data = json.load(f)
        filename = list(data.keys())
        caption = list(data.values())
    fn_list.extend(filename)
    caption_list.extend(caption)

image_real_df = pd.DataFrame(data = {"filename":fn_list,"caption":caption_list})

#drop duplicates 
image_real_df = image_real_df.drop_duplicates(subset = "filename")

# Reading image caption json (scam)

In [56]:
json_dir = 'final caption model/christine_scam'
json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

fn_list = []
caption_list = []
for file in file_list:
    with open(file) as f:
        data = json.load(f)
        filename = list(data.keys())
        caption = list(data.values())
    fn_list.extend(filename)
    caption_list.extend(caption)

image_scam_df = pd.DataFrame(data = {"filename":fn_list,"caption":caption_list})

In [57]:
##concat scam and real profiles
scam_and_real = pd.concat([scamdf[["username","images","y"]], realdf[["username","images","y"]]], ignore_index=True)
#scam_and_real.info()

In [58]:
scam_and_real.head()

,username,images,y
0,nathangilbert130,nathangilbert1301.jpg,0
1,keneth,keneth1.jpg,0
2,josephmax,josephmax1.jpg,0
3,Esanzab,Esanzab1.jpg,0
4,Jhjh28,Jhjh281.jpg,0


In [43]:
#combine image captions + filename (real and scam) into one df
scam_image_and_real_image = pd.concat([image_real_df,image_scam_df],ignore_index = True)
scam_image_and_real_image.rename(columns = {"filename":"images"},inplace = True)

In [59]:
#left join scam image and real image captions to scam_and_real
final_df = scam_and_real.merge(scam_image_and_real_image,how = 'left', on = 'images')

In [66]:
final_df2 = scam_image_and_real_image.merge(scam_and_real, how = 'left',on = 'images')

In [67]:
final_df2.head()

,images,caption,username,y
0,123canwe.jpg,man in red shirt is standing on bench next to ...,123canwe,1.0
1,123WILFREDO.jpg,man in red shirt is sitting on the edge of river,123WILFREDO,1.0
2,52Jim52.jpg,man in black shirt is standing in front of woo...,52Jim52,1.0
3,Aaron90.jpg,two girls are playing in the grass,Aaron90,1.0
4,Abou.jpg,man in red shirt and black shorts is walking d...,Abou,1.0


In [69]:
final_df2[final_df2["username"].isna()]

,images,caption,username,y
3199,martinlawrence2.jpg,man in black shirt is standing in front of store,NaN,NaN
3202,activecarlina1.jpg,man in black shirt is standing in front of crowd,NaN,NaN
3203,activecarlina2.jpg,two girls are playing in the grass,NaN,NaN
3204,activecarlina3.jpg,man in red shirt is standing in front of crowd,NaN,NaN
3205,activecarlina4.jpg,two girls are playing hide and smile,NaN,NaN
...,...,...,...,...
9803,Valery5.jpg,man in red shirt is sitting on bench in front ...,NaN,NaN
9805,franklin1.jpg,man in black shirt is standing in front of wom...,NaN,NaN
9806,franklin2.jpg,man in black shirt is standing in front of wom...,NaN,NaN
9808,cynthia2141.jpg,two girls are sitting on bench in front of the...,NaN,NaN


In [60]:
final_df.head()

,username,images,y,caption
0,nathangilbert130,nathangilbert1301.jpg,0,man in black shirt is standing in front of wom...
1,keneth,keneth1.jpg,0,man in black shirt is standing in front of she...
2,josephmax,josephmax1.jpg,0,NaN
3,Esanzab,Esanzab1.jpg,0,NaN
4,Jhjh28,Jhjh281.jpg,0,man in black shirt is standing in front of wom...


In [61]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22066 entries, 0 to 22065
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   username  22066 non-null  object
 1   images    22066 non-null  object
 2   y         22066 non-null  int64 
 3   caption   4751 non-null   object
dtypes: int64(1), object(3)
memory usage: 862.0+ KB


In [62]:
final_df.dropna(inplace = True)

In [63]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4751 entries, 0 to 22063
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   username  4751 non-null   object
 1   images    4751 non-null   object
 2   y         4751 non-null   int64 
 3   caption   4751 non-null   object
dtypes: int64(1), object(3)
memory usage: 185.6+ KB


In [ ]:
finaldf - 4751 profiles, 1000+ scam, 3000+ real



In [75]:
#try combine finaldf with combined df
final_df3 = combined_df.merge(final_df[["username","caption"]],how = "left",on = "username")

In [76]:
final_df3.head()

,username,age,occupation,status,gender,filtered sentence,y,caption
0,sandralee,19,student,single,female,Im really chill person looking someone chill I...,0,two girls are playing in the tub
1,dayana16,19,estudiante,single,female,white Bisexual Other nonsmoker occasional drin...,0,two girls are playing in the grass
2,Denn_,19,Estudiante,single,female,hispanic Bisexual Atheist nonsmoker never Fun ...,0,two girls are playing in the grass
3,Gabrielita,19,Estudiante,single,female,hispanic Bisexual Christian nonsmoker never Se...,0,NaN
4,Luisa5401,19,Estudiante,single,female,hispanic Straight Other nonsmoker never Fun Fr...,0,two girls are playing in the grass


In [77]:
final_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7934 entries, 0 to 7933
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   username           7934 non-null   object
 1   age                7934 non-null   int64 
 2   occupation         7934 non-null   object
 3   status             7934 non-null   object
 4   gender             7934 non-null   object
 5   filtered sentence  7934 non-null   object
 6   y                  7934 non-null   int64 
 7   caption            4382 non-null   object
dtypes: int64(2), object(6)
memory usage: 557.9+ KB


In [78]:
final_df3.dropna(inplace = True)

In [103]:
#use final df3 - create model with both description + caption 
# features
X = final_df3['caption']

# target
y = final_df3['y']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 424)

In [87]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [105]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(X)
xtrain_tfidf =  tfidf_vect.transform(X_train)
xvalid_tfidf =  tfidf_vect.transform(X_test)

# Naive Bayes classifier for image captions

In [106]:
#random oversampling
text_clf = MultinomialNB()
ros = RandomOverSampler(random_state=42)

X_train_ros, y_train_ros= ros.fit_resample(xtrain_tfidf, y_train)
text_clf = text_clf.fit(X_train_ros, y_train_ros)
predictions = text_clf.predict(xvalid_tfidf)

In [89]:
#evaluating the naive bayes classifier - oversampling
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("random oversampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

random oversampling
Accuracy: 0.7263397947548461
Precision: 0.535796766743649
recall: 0.8560885608856088
f1_score: 0.6590909090909091


In [95]:
#random undersampling
text_clf = MultinomialNB()
rus = RandomUnderSampler(random_state=42)

X_train_rus, y_train_rus= rus.fit_resample(xtrain_tfidf, y_train)
text_clf = text_clf.fit(X_train_rus, y_train_rus)
predictions = text_clf.predict(xvalid_tfidf)

In [96]:
#evaluating the naive bayes classifier - undersampling
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("random undersampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

random undersampling
Accuracy: 0.7263397947548461
Precision: 0.5364705882352941
recall: 0.8413284132841329
f1_score: 0.6551724137931034


# svm classifier

In [93]:
from sklearn.linear_model import SGDClassifier

#random oversampling
text_clf_svm = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42)

ros = RandomOverSampler(random_state=42)

X_train_ros, y_train_ros= ros.fit_resample(xtrain_tfidf, y_train)
text_clf_svm = text_clf_svm.fit(X_train_ros, y_train_ros)
predictions = text_clf_svm.predict(xvalid_tfidf)

In [94]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("random oversampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

random oversampling
Accuracy: 0.7457240592930444
Precision: 0.5524017467248908
recall: 0.933579335793358
f1_score: 0.6941015089163236


In [97]:
#random undersampling
text_clf_svm = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42)

rus = RandomUnderSampler(random_state=42)

X_train_rus, y_train_rus= rus.fit_resample(xtrain_tfidf, y_train)
text_clf_svm = text_clf_svm.fit(X_train_rus, y_train_rus)
predictions = text_clf_svm.predict(xvalid_tfidf)

In [98]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("random undersampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

random undersampling
Accuracy: 0.7457240592930444
Precision: 0.5535714285714286
recall: 0.915129151291513
f1_score: 0.6898470097357441


# logistic regression classifier

In [109]:
from sklearn.linear_model import LogisticRegression
#random oversampling
clf_lg = LogisticRegression()  

ros = RandomOverSampler(random_state=42)

X_train_ros, y_train_ros= ros.fit_resample(xtrain_tfidf, y_train)
clf_lg = clf_lg.fit(X_train_ros, y_train_ros)
lg_predictions = clf_lg.predict(xvalid_tfidf)

In [110]:
conf_matrix = confusion_matrix(y_test, lg_predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("random oversampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

random oversampling
Accuracy: 0.750285062713797
Precision: 0.5601851851851852
recall: 0.8929889298892989
f1_score: 0.6884779516358465


In [111]:
#random undersampling
clf_lg = LogisticRegression()  

rus = RandomUnderSampler(random_state=42)

X_train_rus, y_train_rus= rus.fit_resample(xtrain_tfidf, y_train)
clf_lg = clf_lg.fit(X_train_rus, y_train_rus)
lg_predictions = clf_lg.predict(xvalid_tfidf)

In [112]:
conf_matrix = confusion_matrix(y_test, lg_predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("random undersampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

random undersampling
Accuracy: 0.7525655644241733
Precision: 0.5627906976744186
recall: 0.8929889298892989
f1_score: 0.6904422253922967
